In [1]:
#从selenium导入浏览器驱动
from selenium import webdriver
#导入浏览器驱动设置选项
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
#导入网页解析库
from bs4 import BeautifulSoup
#导入时间库,利用time.time()防止爬虫访问过于频繁被禁止访问
import time
#导入pandas数据分析库,生成dataframe
import pandas as pd
import pickle
import re
from china_cities import *
import csv
import requests, lxml
import json
import pinyin.cedict

In [ ]:
chinese_company_name = '苏宁易购集团股份有限公司'

In [3]:
ENV = 'dev'
ENV_FILENAME = 'prod.env' if ENV == 'prod' else 'dev.env'
AIRTABLE_SHEET_NAME = 'Company Data ENTRY TABLE' if ENV == 'prod' else 'qcc_scrape'

In [4]:
env_vars = {}

with open(ENV_FILENAME) as f:
    for line in f:
        if line.startswith('#') or not line.strip():
            continue
        key, value = line.strip().split('=')
        env_vars[key] = value

AIRTABLE_TOKEN = env_vars["AIRTABLE_TOKEN"]
AIRTABLE_BASE_ID = env_vars["AIRTABLE_BASE_ID"]
AIRTABLE_URL = f"https://api.airtable.com/v0/{AIRTABLE_BASE_ID}"

In [5]:
def get_registered_year(str_table):
    str_table_year = re.compile(r'([1|2]\d\d\d)[-](0[1-9]|1[012])[-](0[1-9]|[12][0-9]|3[01])')
    iterator = str_table_year.findall(str_table)
    if len(list(iterator)) > 0:
         return list(iterator)[0][0]
    elif "年" in str_table:
        str_table_year = re.compile(r'([1|2]\d\d\d)[年](\d+)[月](\d+)[日]')
        iterator = str_table_year.findall(str_table)
        return list(iterator)[0][0]

In [6]:
def get_headquarter(str_table):
    headquarters = {
        '开曼群岛': 'Cayman Islands',
        '開曼群島':'Cayman Islands',
        '英属维尔京群岛': 'British Virgin Islands'
    }
    with open('chinese_cities.csv', newline='') as csvfile:
        headquarter_reader = csv.reader(csvfile, delimiter=',')
        for en_city, cn_city, en_province in headquarter_reader:
            if cn_city in ['澳门', '澳門', '北京市', '重庆市', '上海市', '天津市']:
                headquarters[cn_city] = "{}, China".format(en_city)
            elif cn_city in ['香港']:
                headquarters[cn_city] = "{} SAR, China".format(en_city)
            else:
                headquarters[cn_city] = "{}, {}, China".format(en_city, en_province)

    en_headquarter = [headquarter for cn_city, headquarter in headquarters.items() if cn_city in str_table]
    if '非香港' in str_table:
        en_headquarter.remove('Hong Kong SAR, China')
    if 'Cayman Islands' in en_headquarter:
        return 'Cayman Islands'
    if 'British Virgin Islands' in en_headquarter:
        return 'British Virgin Islands'
    if len(en_headquarter) == 0:
        return ''
    else:
        return en_headquarter[0]

In [7]:
driver = webdriver.Chrome() #实例化一个浏览器
driver.get('https://www.qcc.com/')
print('请在打开的网页，扫码登录！')
time.sleep(3)
new_cookies = driver.get_cookies() #获取最新的cookies
pickle.dump( new_cookies, open("cookies.pkl","wb"))
driver.quit() #关闭浏览器

请在打开的网页，扫码登录！


In [8]:
'''若取消下方被注释的代码可静默运行浏览器，不会显示页面，仅在后台运行'''
chrome_options=Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
driver = webdriver.Chrome()           #实例化一个谷歌浏览器对象
driver.get("https://www.qcc.com/")   #浏览器打开企查查网站

cookies = pickle.load(open("cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
driver.get("https://www.qcc.com/")
driver.refresh()                #自动刷新页面，请检查是否已经自动登录账号

In [16]:
def get_company_table(company):
    time.sleep(3.2)
    driver.find_element(By.XPATH, "//input[contains(@id,'searchKey')]").clear()
    driver.find_element(By.XPATH, "//input[contains(@id,'searchKey')]").send_keys(company)
    driver.find_element(By.XPATH, "//button[@class='btn btn-primary']").click()
    time.sleep(3)
    bs = BeautifulSoup(driver.page_source,'html.parser')  #将加载好的网页用BeautifulSoup解析成文本
    search_company_list_table = bs.find_all('table')[0].find_all('tr')
    search_company_list_data = []
    
    for table_row in search_company_list_table:
        row_data = []
        for column, table_cell in enumerate(table_row):
            try:
                if column < 2:
                    continue
                cell_content = table_cell.text.strip().replace('最近浏览','').split('\n',1)[0].split(' 存续',1)[0].strip()
                row_data.append(cell_content.split(' ',1)[0])
            except:
                continue
        search_company_list_data.append(row_data)
        
    dataFrame = pd.DataFrame(data = search_company_list_data[:], columns = search_company_list_data[0])
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(dataFrame)
        
    company_name = str(dataFrame.loc[0:0]).strip().split('\n')[0]
    print('A:', company_name)
    print('B:', company)
    if company_name == company:
        wait = WebDriverWait(driver, 3)
        links = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href,'https://www.qcc.com/firm/')]")))
        driver.execute_script("arguments[0].target='_self';", links[0])
        driver.find_element(By.XPATH, "//a[contains(@href,'https://www.qcc.com/firm/')]").click()
        time.sleep(3)
        bs = BeautifulSoup(driver.page_source,'html.parser')  #将加载好的网页用BeautifulSoup解析成文本
        # print('漂亮的汤:', bs)

        html_table = bs.find("table")
        str_table = str(html_table)
        # print('基本信息:', str_table)
        return str_table
    else:
        return False

In [19]:
company_table = get_company_table(chinese_company_name)
print(company_table)

,,,苏宁易购集团股份有限公司
0,,,苏宁易购集团股份有限公司
1,,,十大股东
2,,,苏宁易购集团股份有限公司苏宁采购中心
3,,,苏宁易购集团股份有限公司上海苏宁互联分公司
4,,,苏宁易购集团股份有限公司北京苏宁互联分公司
5,,,苏宁易购集团股份有限公司南京苏宁互联分公司
6,,,苏宁易购集团股份有限公司服务分公司
7,,,苏宁易购集团股份有限公司成都苏宁互联分公司
8,,,苏宁易购集团股份有限公司南昌苏宁互联分公司
9,,,苏宁易购集团股份有限公司保定苏宁互联分公司


A: 苏宁易购集团股份有限公司
B: 苏宁易购集团股份有限公司
<table class="ntable"><tr><td class="tb" width="13%">统一社会信用代码 <i aria-label="icon: icon-icon_zhushi" class="app-glossary-info glossary-info anticon anticon-icon-icon_zhushi aicon aicon-zhushi"><svg aria-hidden="true" class="" fill="currentColor" focusable="false" height="1em" width="1em"><use xlink:href="#icon-icon_zhushi"></use></svg></i></td> <td style="padding-right: 0px;" width="21%"><div class="app-copy-box copy-hover-item" data-v-7a4909d8=""><span class="copy-value" data-v-7a4909d8="">91320000608950987L</span> <span class="app-copy copy-button-item" data-v-7a4909d8=""><div class="base_copy" data-v-7a4909d8=""><i aria-label="icon: icon-icon_fuzhi" class="anticon anticon-icon-icon_fuzhi aicon aicon-fuzhi" data-v-7a4909d8=""><svg aria-hidden="true" class="" fill="currentColor" focusable="false" height="1em" width="1em"><use data-v-7a4909d8="" xlink:href="#icon-icon_fuzhi"></use></svg></i> <span data-v-7a4909d8="">复制</span></div></span> <textarea class

In [9]:
def save_in_json(input, filename):        
    with open(filename, 'w') as jsonfile:
        jsonfile.write(json.dumps(input, indent=4))

In [10]:
def get_airtable_data(sheet_name, offset=None):
    url = f"{AIRTABLE_URL}/{sheet_name}"
    headers = {
        'Authorization': f'Bearer {AIRTABLE_TOKEN}',
        'Content-Type': 'application/json'
    }
    params = {}
    if offset:
        params["offset"] = offset
    
    response = requests.request("GET", url, headers=headers, params=params)
    return response

In [11]:
#读取airtable
sheet_name = AIRTABLE_SHEET_NAME
airtable_response = get_airtable_data(sheet_name).json()
all_records = []
records = get_airtable_data(sheet_name)
all_records.extend(records.json()['records'])

while "offset" in records.json():
    records = get_airtable_data(sheet_name, records.json()["offset"])
    all_records.extend(records.json()['records'])

save_in_json(all_records, 'get.json')

# Ownership for Public Company

In [13]:
#点上市信息
element = driver.find_element(By.XPATH, "//a[starts-with(@href,'/firm/')]")
print(element.get_attribute("innerHTML"))
element.click()
time.sleep(3)
#重新截取网页
bs = BeautifulSoup(driver.page_source,'html.parser')

<h2 class="">
            上市信息
            <span class="count">201</span></h2>


In [14]:
#找十大股东
all_elements = driver.find_elements(By.XPATH, "//span[@class='name']/a")
top_ten_shareholders = []

for index, element in enumerate(all_elements):
    top_ten_shareholders.append(element.get_attribute("innerHTML"))

In [15]:
#十大股东的表格
shareholders_table = bs.find_all('table')[7].find_all('tr')
shareholders_data = []

for table_row in shareholders_table:
    row_data = []
    for column, table_cell in enumerate(table_row):
        try:
            if column < 4:
                continue
            if column >= 4:
                cell_content = table_cell.text.strip().split('\n',1)[0].split(' 股',1)[0].split(' 大股东',1)[0].split(' 有股权质押',1)[0]
            if column == 6 and cell_content != '持股数(股)':
                cell_content = "{:,}".format(int(cell_content))
            if column == 9 and cell_content != '增减(股)'and cell_content != '不变':
                cell_content = "{:,}".format(int(cell_content))
            row_data.append(cell_content)
        except:
            continue
    shareholders_data.append(row_data)

dataFrame = pd.DataFrame(data = shareholders_data[1:], columns = shareholders_data[0])
dataFrame.to_csv('苏宁易购集团股份有限公司_shareholders.csv')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(dataFrame)

,股东名称,股份类型,持股数(股),持股比例,最终受益股份,增减(股),变动比例
0,淘宝（中国）软件有限公司,流通A股,"1,861,076,927",19.99%,20.0482%,不变,-
1,张近东,"流通A股,限售流通A股","1,640,181,431",17.62%,19.8465%,不变,-
2,江苏新新零售创新基金二期（有限合伙）,流通A股,"1,578,696,146",16.96%,16.96%,不变,-
3,江苏新新零售创新基金（有限合伙）,流通A股,"520,000,000",5.59%,5.59%,不变,-
4,李 李松强,流通A股,"281,526,374",3.02%,3.02%,不变,-
5,苏宁控股集团有限公司,流通A股,"254,411,429",2.73%,2.73%,不变,-
6,香港中央结算有限公司 香港,流通A股,"205,216,022",2.20%,-,"8,549,946",4.35%
7,苏宁电器集团有限公司,流通A股,"129,448,134",1.39%,1.39%,不变,-
8,金明,流通A股,"125,001,165",1.34%,1.34%,不变,-
9,陈金凤,流通A股,"73,039,097",0.78%,0.78%,不变,-
